<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/TF_IDF_UP25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cargar la base

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
base=pd.read_csv('/content/drive/MyDrive/base_formateada.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Se incorporan las stopwords sugeridas por Federico
stopwords.extend(['señor', 'señora', 'mucha', 'gracia', 'año', 'pasado', 'cada', 'vez', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis' ,'siete', 'ocho', 'nueve', 'diez'])
stopwords.extend(['hoy', 'aquí', 'primer', 'lugar', 'primera', 'segunda', 'primero', 'segundo', 'siguiente', 'tercer'])
stopwords.extend(['convergencia', 'esquerra', 'republicana', 'grupo', 'parlamentaria', 'parlamentario', 'partido'])
stopwords.extend(['enmienda', 'votación', 'favor', 'abstención', 'ley', 'real', 'decreto', 'decretoley', 'resultado', 'voto', 'sé' ,'sí', 'silencio' ,'favor'])
stopwords.extend(['usted', 'señoría', 'presidente', 'presidenta', 'ministro', 'orden' , 'día', 'palabra', 'petición', 'posición', 'punto', 'vista', 'sesión', 'baldoví', 'duran', 'turno'])
stopwords.extend(['hacer', 'frente', 'puede' ,'ser', 'va', 'voy', 'decir'])
stopwords.extend(['millón', 'euro', 'emitido', 'efectuada', 'dio', 'comienzo', 'partido', 'queda', 'quedan', 'rechazada', 'aceptada', 'comienza', 'usted', 'sabe', 'abstención', 'diputado', 'gobierno'])
stopwords.extend(['continuación', 'votamos', 'telemático', 'republicanaizquierda', 'unidainiciativa', 'puede', 'bien', 'propuesta' ,'abstencion', 'mayoría', 'absoluta', 'pregunta', 'don', 'vamos', 'votar', 'llevar', 'cabo', 'millón', 'muchas', 'gracias'])

In [5]:
# Se incorporan los nombres de los oradores como stopwords
from collections import Counter
nombres=[]
for i in base.namey.unique().tolist(): nombres.extend(str(i).split(' '))
nombres = [x.strip(' ') for x in nombres]
nombres = [x.strip(' †') for x in nombres]
nombres = [x.strip(',') for x in nombres]
nombres=list(Counter(nombres))
nombres.remove('')

In [6]:
stopwords.extend(nombres)

# TF-IDF

In [7]:
# El hiperparámetro min_df del TF-IDF elimina las palabras con menor frecuencia. Por ejemplo, las palabras que aparecen en menos que el 75% de los discursos. 
# A continuación se adopta un enfoque distinto: Se genera un vocabulario con todos los bigramas, y luego se elimina el 75% de los bigramas con menor TF-IDF. 
# Es decir que entre los 8 millones de bigramas se seleccionan los 2 millones de mayor TF-IDF. 

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(ngram_range=[2,2], stop_words=stopwords)
bow = vect.fit_transform(base['tokens'])
total_features = len(vect.vocabulary_)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abad', 'abascal', 'abeilhe', 'abejón', 'abel', 'aburto', 'acebes', 'acedo', 'acevedo', 'achutegui', 'acosta', 'acín', 'acítores', 'adela', 'adelaida', 'adolfo', 'adriana', 'adriano', 'adrover', 'adsuar', 'adámez', 'adán', 'agius', 'agramunt', 'aguado', 'aguerri', 'aguilar', 'aguilera', 'aguiriano', 'aguirre', 'agustín', 'ahuja', 'aizpún', 'aja', 'alarcón', 'alba', 'albaladejo', 'albendea', 'alberdi', 'albero', 'alberto', 'albiach', 'albiñana', 'alborch', 'albéniz', 'alcalde', 'alcaraz', 'alcañiz', 'alconchel', 'alcover', 'alcázar', 'aldea', 'aledo', 'alegre', 'alegría', 'alejandro', 'alfonso', 'alfred', 'alfredo', 'alférez', 'alicia', 'alierta', 'alique', 'allende', 'almagro', 'almaraz', 'almunia', 'alonso', 'alvear', 'alzaga', 'alén', 'alós', 'amador', 'amann', 'amarillo', 'amate', 'amelia'

In [13]:
total_features

8228192

In [10]:
sum_words = bow.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1])
vocabulary, _ = zip(*words_freq[:int(total_features * 0.75)])
less_vocabulary = list(vocabulary)

In [11]:
voc=vect.vocabulary_.keys()-less_vocabulary

In [12]:
len(voc)

2057048

In [14]:
cv=TfidfVectorizer(ngram_range=[2,2], vocabulary=voc)
vec = cv.fit(base['tokens'])

In [15]:
matrix = vec.transform(base['tokens'])

In [16]:
matrix.shape

(334421, 2057048)

# Número óptimo de clustes

In [ ]:
# LDA en Sklearn tiene un resultado de score correspondiente al log-likelihood. En base a ese criterio se identifica el número óptimo de clusters. 

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

search_params = {'n_components': [20, 40, 60, 80]}
model = LatentDirichletAllocation(max_iter=500, learning_method='online', learning_offset=50.,random_state=0)
gridsearch = GridSearchCV(model,
                          param_grid=search_params,
                          n_jobs=-1,
                          verbose=1)
gridsearch.fit(matrix)
best_lda = gridsearch.best_estimator_